In [5]:
import sys
sys.path.append("../../src")
import os
import datetime
import pandas as pd
import numpy as np
from example_pendulum import get_pendulum_data
from sindy_utils import library_size
from training import train_network
import tensorflow as tf
tf.compat.v1.disable_eager_execution()


# Generate data

In [6]:
training_data = get_pendulum_data(100, True)
validation_data = get_pendulum_data(10)

Generating pendulum data with noise
Generating pendulum data


# Set up model and training parameters

In [7]:
params = {}

params['input_dim'] = training_data['x'].shape[-1]
params['latent_dim'] = 1
params['model_order'] = 2
params['poly_order'] = 3
params['include_sine'] = True
params['library_dim'] = library_size(2*params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_x'] = 5e-4
params['loss_weight_sindy_z'] = 5e-5
params['loss_weight_sindy_regularization'] = 1e-5

params['activation'] = 'sigmoid'
params['widths'] = [128,64,32]

# training parameters
params['epoch_size'] = training_data['x'].shape[0]
params['batch_size'] = 1000
params['learning_rate'] = 1e-4

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 100

# training time cutoffs
params['max_epochs'] = 5001
params['refinement_epochs'] = 1001

# Run training experiments

In [8]:
num_experiments = 10
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))

    params['save_name'] = 'pendulum_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")

    tf.compat.v1.reset_default_graph()

    results_dict = train_network(training_data, validation_data, params)
    df = df.append({**results_dict, **params}, ignore_index=True)

df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

EXPERIMENT 0
TRAINING
Epoch 0
   training loss 0.010535256005823612, (0.007856007, 0.32767388, 5.3058076, 0.9961781)
   validation loss 0.017946472391486168, (0.008318123, 0.32703128, 19.204071, 0.9961781)
decoder loss ratio: 0.991696, decoder SINDy loss  ratio: 1.000000
Epoch 100
   training loss 0.009700222872197628, (0.0070414655, 6.4593223e-06, 5.3058066, 0.58533174)
   validation loss 0.017183832824230194, (0.0075759427, 2.9059416e-05, 19.204071, 0.58533174)
decoder loss ratio: 0.903212, decoder SINDy loss  ratio: 1.000000
Epoch 200
   training loss 0.009680281393229961, (0.007025956, 0.00021856307, 5.3058066, 0.14108706)
   validation loss 0.01717873476445675, (0.007575241, 0.0009545783, 19.204073, 0.14108706)
decoder loss ratio: 0.903128, decoder SINDy loss  ratio: 1.000000
Epoch 300
   training loss 0.009680494666099548, (0.007027168, 0.0007444198, 5.3058066, 0.038596053)
   validation loss 0.017174245789647102, (0.0075716334, 0.0038061035, 19.204073, 0.038596053)
decoder loss 

AttributeError: 'DataFrame' object has no attribute 'append'